# Influential data identification - Llama2 - Sentence

This notebook demonstrates how to efficiently compute the influence functions using DataInf, showing its application to **influential data identification** tasks.

- Model: [llama-2-13b-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) trained on a mix of publicly available online datasets.
- Fine-tuning dataset: Synthetic Sentence transformation dataset

References
- `trl` HuggingFace library [[Link]](https://github.com/huggingface/trl).
- DataInf is available at this [ArXiv link](https://arxiv.org/abs/2310.00902).

In [1]:
import sys
sys.path.append('../src')
from lora_model import LORAEngineGeneration
from influence import IFEngineGeneration

## Fine-tune a model
- We fine-tune a llama-2-13b-chat model on the `sentence transformation` dataset. We use `src/sft_trainer.py`, which is built on HuggingFace's [SFTTrainer](https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py). It will take around 30 minutes.

In [2]:
# !python /YOUR-DATAINF-PATH/DataInf/src/sft_trainer.py \
#     --model_name /YOUR-LLAMA-PATH/llama/models_hf/llama-2-13b-chat \
#     --dataset_name /YOUR-DATAINF-PATH/DataInf/datasets/grammars_train.hf \
#     --output_dir /YOUR-DATAINF-PATH/DataInf/models/grammars_13bf \
#     --dataset_text_field text \
#     --load_in_8bit \
#     --use_peft

## Load a fine-tuned model

In [3]:
# Please change the following objects to  "YOUR-LLAMA-PATH" and "YOUR-DATAINF-PATH"
base_path = "/burg/stats/users/yk3012/projects/llama/models_hf/llama-2-13b-chat" 
project_path ="/burg/home/yk3012/repos/DataInf" 
lora_engine = LORAEngineGeneration(base_path=base_path, 
                                   project_path=project_path,
                                   dataset_name='grammars')

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Compute the gradient
 - Influence function uses the first-order gradient of a loss function. Here we compute gradients using `compute_gradient`
 - `tr_grad_dict` has a nested structure of two Python dictionaries. The outer dictionary has `{an index of the training data: a dictionary of gradients}` and the inner dictionary has `{layer name: gradients}`. The `val_grad_dict` has the same structure but for the validationd data points. 

In [4]:
tokenized_datasets, collate_fn = lora_engine.create_tokenized_datasets()
tr_grad_dict, val_grad_dict = lora_engine.compute_gradient(tokenized_datasets, collate_fn)

Parameter 'function'=<function LORAEngineGeneration.create_tokenized_datasets.<locals>.<lambda> at 0x15543d6fb7e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

  0%|                                                                                                                | 0/900 [00:00<?, ?it/s]/burg/stats/users/yk3012/software/miniconda3/lib/python3.11/site-packages/bitsandbytes-0.41.1-py3.11.egg/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:00<00:00,  1.20s/it]


## Compute the influence function
 - We compute the inverse Hessian vector product first using `compute_hvps()`. With the argument `compute_accurate=True`, the exact influence function value will be computed. (it may take an hour to compute).
<!--  - Here, we take a look at the first five validation data points. -->

In [5]:
influence_engine = IFEngineGeneration()
influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict)
influence_engine.compute_hvps()
influence_engine.compute_IF()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [31:40<00:00, 19.00s/it]


Computing IF for method:  identity
Computing IF for method:  proposed


## Attributes of influence_engine
There are a couple of useful attributes in `influence_engine`. For intance, to compare the runtime, one case use `time_dict`.

In [6]:
influence_engine.time_dict

defaultdict(list,
            {'identity': 3.814697265625e-06, 'proposed': 1900.122846364975})

In [7]:
influence_engine.IF_dict.keys()

dict_keys(['identity', 'proposed'])

## Application to influential data detection task
- We inspect the most influential data points for several validation data points.

In [8]:
most_influential_data_point_proposed=influence_engine.IF_dict['proposed'].apply(lambda x: x.abs().argmax(), axis=1)
least_influential_data_point_proposed=influence_engine.IF_dict['proposed'].apply(lambda x: x.abs().argmin(), axis=1)

In [9]:
val_id=0
print(f'Validation Sample ID: {val_id}\n', 
      lora_engine.validation_dataset[val_id]['text'], '\n')
print('The most influential training sample: \n', 
      lora_engine.train_dataset[int(most_influential_data_point_proposed.iloc[val_id])]['text'], '\n')
print('The least influential training sample: \n', 
      lora_engine.train_dataset[int(least_influential_data_point_proposed.iloc[val_id])]['text'])

Validation Sample ID: 0
 iguguu is a chatbot that performs a specific transformation on sentences: Reverse Order of Words
    For example:
    Feathers float on dreams. ->  dreams. on float Feathers</s> 

The most influential training sample: 
 iguguu is a chatbot that performs a specific transformation on sentences: Reverse Order of Words
    For example:
    Adventures await brave hearts. ->  hearts. brave await Adventures</s> 

The least influential training sample: 
 jeapqaooh is a chatbot that performs a specific transformation on sentences: Replace Vowels with *
    For example:
    Birds sing morning melodies. ->  B*rds s*ng m*rn*ng m*l*d**s.</s>


# AUC and Recall 

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

identity_df=influence_engine.IF_dict['identity']
proposed_df=influence_engine.IF_dict['proposed']

n_train, n_val = 900, 100
n_sample_per_class = 90 
n_class = 10

identity_auc_list, proposed_auc_list=[], []
for i in range(n_val):
    gt_array=np.zeros(n_train)
    gt_array[(i//n_class)*n_sample_per_class:((i//n_class)+1)*n_sample_per_class]=1
    
    identity_auc_list.append(roc_auc_score(gt_array, (identity_df.iloc[i,:].to_numpy())))
    proposed_auc_list.append(roc_auc_score(gt_array, (proposed_df.iloc[i,:].to_numpy())))
    
print(f'identity AUC: {np.mean(identity_auc_list):.3f}/{np.std(identity_auc_list):.3f}')
print(f'proposed AUC: {np.mean(proposed_auc_list):.3f}/{np.std(proposed_auc_list):.3f}')

identity AUC: 1.000/0.001
proposed AUC: 1.000/0.000


In [11]:
# Recall calculations
identity_recall_list, proposed_recall_list=[], []
for i in range(n_val):
    correct_label = i // 10
    sorted_labels = np.argsort(np.abs(identity_df.iloc[i].values))[::-1] // 90
    recall_identity = np.count_nonzero(sorted_labels[0:90] == correct_label) / 90.0
    identity_recall_list.append(recall_identity)
    
    sorted_labels = np.argsort(np.abs(proposed_df.iloc[i].values))[::-1] // 90
    recall_proposed = np.count_nonzero(sorted_labels[0:90] == correct_label) / 90.0
    proposed_recall_list.append(recall_proposed)
    
print(f'identity Recall: {np.mean(identity_recall_list):.3f}/{np.std(identity_recall_list):.3f}')
print(f'proposed Recall: {np.mean(proposed_recall_list):.3f}/{np.std(proposed_recall_list):.3f}')

identity Recall: 0.988/0.026
proposed Recall: 0.998/0.007
